In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
import datetime
from tqdm import tqdm, trange 

import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data_1_df = pd.read_excel('./data/data_1.xlsx')
raw_data_2_df = pd.read_excel('./data/data_2.xlsx')
raw_data_3_df = pd.read_excel('./data/data_3.xlsx')

In [3]:
raw_data_1_df.head()

,时间,Unnamed: 1,Unnamed: 2,时间间隔,GPS车速,加速度,X轴加速度,Y轴加速度,Z轴加速度,经度,纬度,发动机转速,扭矩百分比,瞬时油耗,油门踏板开度,空燃比,发动机负荷百分比,进气流量
0,2017/12/18 13:42:13.000.,2017/12/18 13:42:13,NaN,NaN,0.0,NaN,0.0,-0.396,-0.900,119.367781,25.992420,775,18,58.02,0.0,0.1465,22,2.30
1,2017/12/18 13:42:14.000.,2017/12/18 13:42:14,00:00:01,1.0,0.0,0.0,0.0,-0.378,-0.882,119.367783,25.992418,775,17,60.30,0.0,0.1465,21,2.39
2,2017/12/18 13:42:15.000.,2017/12/18 13:42:15,00:00:01,1.0,0.0,0.0,0.0,-0.396,-0.882,119.367785,25.992416,775,17,55.24,0.0,0.1464,22,2.19
3,2017/12/18 13:42:16.000.,2017/12/18 13:42:16,00:00:01,1.0,0.0,0.0,0.0,-0.378,-0.900,119.367785,25.992416,762,17,55.75,0.0,0.1471,21,2.21
4,2017/12/18 13:42:17.000.,2017/12/18 13:42:17,00:00:01,1.0,0.0,0.0,0.0,-0.396,-0.882,119.367785,25.992416,762,16,56.00,0.0,0.1471,21,2.22


## 数据预处理

### 预处理方案

问题汇总：
    1. 数据是要创建成新的运动学片段数据，那么先前的特征'GPS车速', '加速度', 'X轴加速度',
       'Y轴加速度', 'Z轴加速度', '经度', '纬度', '发动机转速', '扭矩百分比', '瞬时油耗', '油门踏板开度', '空燃比',
       '发动机负荷百分比', '进气流量'都不要了吗
    2. 对于数据填充还是不太理解，需要讲个例子进行理解
    3. 怠速时间的问题，删除超出180秒的怠速记录操作不会对后面的特征（怠速时间）造成影响？
    4. 运动学片段划分原则里，加速度是怎么理解？

数据预处理
    1. 对时间间隔在1分钟以内的数据采用插值法进行填补（只对车速），而对时间间隔超出1分钟的数据不处理
        * 公式讲解的不明确，不知道是用哪个值进行填补
    2. 加减速异常的数据
        * 加速度大于4的数据，用前5秒车速的均值去替换此车速（也可替换为前1秒的车速）  万一时间间隔是5分钟，没有前1秒和前5秒呢
            * 这里有个问题，发现1637的加速度为4.22，但是前5秒车速都为0，后5秒都为加速状态，这样直接赋值前5秒均值会造成数据出问题
        * 加速度小于-8的数据，用前5秒车速的均值去替代
    3. 长时间堵车和断断续续低速行驶情况
        * 选取连续10秒车速大于0小于10的数据，速度替换为0  如果连续15秒都是0~10，那么是15秒速度全替换为0？
    4. 怠速时间
        * 怠速时间超出180秒且车速为0的情况，此时选取连续超出180秒车速为0的数据，保留后180秒数据，剔除前面的，即删减冗余数据


运动学片段划分原则：
    1. 速度从0开始
    2. 包含加速度大于0.1，小于-0.1，大于-0.1小于0.1的情况  （这个加速度到底是分了哪几种情况）
    3. 速度以0结束
    4. 片段时长大于20秒
    5. 划分好片段后，若片段中的时间不连续，则剔除该片段  （数据不是依靠时间流进行的吗？为何会出现时间不连续的情况，因为前面怠速删除？）

特征参数体系构建
    1. 怠速时间   （怠速时间超出180s的都被剔除了，不会影响结果吗？）
    2. 加速时间
    3. 匀速时间
    4. 减速时间
    5. 运行时间    
    6. 运行距离
    7. 最大速度
    
    8. 匀速时间比例
    9. 怠速时间比例
    10. 减速时间比例
    11. 加速时间比例
    12. 速度在0-10的比例
    13. 速度在10-20的比例
    14. 速度在20-30的比例
    15. 速度在30-40的比例
    16. 速度在40-50的比例
    17. 速度在50-60的比例
    18. 速度在60-70的比例
    19. 速度在大于70的比例

### 预处理

#### 为间隔时间在60s以内的数据进行线性填充

In [4]:
def delData(df):
    '''
    @func: 删除不必要数据和更改数据名
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''
    df = df.drop(columns=['Unnamed: 1', 'Unnamed: 2','X轴加速度','Y轴加速度', 'Z轴加速度', '经度', 
                            '纬度', '发动机转速', '扭矩百分比', '瞬时油耗', '油门踏板开度', '空燃比',
                                    '发动机负荷百分比', '进气流量'])
    df = df.drop([0])  # 删除第一行
    return df


def fillData(df):
    '''
    @func: 为间隔时间在60s以内的数据进行线性填充
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''
    df = delData(df)  # 填充前的数据瘦身
    
    # 利用时间间隔去寻找间隔在1分钟以内的数据
    time_interval = df[df['时间间隔'] > 1.000001]
    time_interval = time_interval[time_interval['时间间隔'] <= 60]
    
    def getAcceVelocity(pre_v, v, time_len):
        '''
        @func: 获取加速度
        @params: [pre_v] 上一个时间点速度
        @params: [v] 当前速度
        @params: [time_len] 时长
        @return: [(v - pre_v) / (time_len * 3.6)] 所得到的当前加速度
        '''
        return (v - pre_v) / (time_len * 3.6)

    def getVelocity(pre_v, a, time_len):
        '''
        @func: 获取当前速度
        @params: [pre_v] 上一个时间点速度
        @params: [a] 加速度
        @params: [time_len] 时长
        @return: [pre_v + a * time_len] 所得到的当前速度
        '''
        return pre_v + a * time_len

    def creatDateframe(last_time, time_len, pre_v, a):
        '''
        @func: 创建索要插入的数据，type为DataFrame
        @params: [last_time] 上一个时间点
        @params: [time_len] 时长
        @params: [pre_v] 上一个时间点速度
        @params: [a] 加速度
        @return: [new_df] 新建数据
        '''
        new_df = pd.DataFrame(columns=['时间', '时间间隔', 'GPS车速', '加速度'])
        for idx in range(time_len):
            v_t = getVelocity(pre_v, a, idx+1)
            time_current = str((last_time + datetime.timedelta(seconds=idx+1)).strftime("%Y/%m/%d %H:%M:%S"))+'.000.'
            new_df = new_df.append({'时间':time_current, '时间间隔':1.0, 'GPS车速':v_t, '加速度':a},ignore_index=True)
        return new_df


    add_time = 0
#     i = 0  # i是轮数限定，用来测试code
    for idx in tqdm(time_interval.index,desc='fillData'):
        idx = idx - 1 + add_time
#         i += 1
        now_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        last_time = datetime.datetime.strptime(df.iloc[idx-1]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        time_span = now_time - last_time
        time_span = int(''.join(re.findall(r"\d+\.?\d*",str(time_span))))

        pre_v = df.iloc[idx-1]['GPS车速']  # 初始速速
        v = df.iloc[idx]['GPS车速']    # 当前速度
        a = getAcceVelocity(pre_v, v, time_span)   # 加速度

        # 对DataFrame的指定位置插入数据，插入数据会破坏索引，因此需要实时记录插入条数
        new_df = creatDateframe(last_time, time_span-1, pre_v, a)  # a~b秒内只需要添加b-a+1行数据，但是最后还要更新当前行
        add_time += (time_span - 1) 
        df1 = df.iloc[:idx] # 将数据按指定位置插入（好麻烦的插入方法）
        df2 = df.iloc[idx:]
        df2['时间间隔'][df2.index[0]] = 1.0
        df = df1.append(new_df, ignore_index = True).append(df2, ignore_index = True)

#         if i>5:break
    return df

#### 对加速度>5和减速度<-8的数据进行数值修正

In [5]:
def moreAcce_and_lessDec(df):
    '''
    @func: 对加速度>5和减速度<-8的数据进行数值修正
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''
    # 查看加速度大于4的数据前面是否存在前5秒的数据
    # i = 0 # examine code
    no_5_seconds = []
    for idx in df[df['加速度']>4].index:
    #     i += 1
        last_5_time = datetime.datetime.strptime(df.iloc[idx-5]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        now_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        time_span = now_time - last_5_time
        time_span = int(''.join(re.findall(r"\d+\.?\d*",str(time_span))))
        if time_span != 5: no_5_seconds.append(idx)
    #     if i>5 :break


    # 查看加速度小于-8的数据前面是否存在前5秒的数据
    # i = 0 # examine code
    no_8_seconds = []
    for idx in df[df['加速度']<-8].index:
    #    i += 1
        last_5_time = datetime.datetime.strptime(df.iloc[idx-5]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        now_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        time_span = now_time - last_5_time
        time_span = int(''.join(re.findall(r"\d+\.?\d*",str(time_span))))
        if time_span != 5: no_8_seconds.append(idx)
    #     if i>5 :break

    # 对超加速度数据进行数值修正
    for idx in tqdm(df[df['加速度']>4].index,desc='moreAcce_and_lessDec for maorAcce'):
        if idx in no_5_seconds: 
            df.iloc[idx]['GPS车速'] = df.iloc[idx-1]['GPS车速']
        df['GPS车速'][idx] = np.average(df.iloc[idx-5:idx]['GPS车速'].values)

    # 对超减速度数据进行数值修正
    for idx in tqdm(df[df['加速度']<-8].index,desc='moreAcce_and_lessDec for lessDec'):
        if idx in no_8_seconds: 
            df.iloc[idx]['GPS车速'] = df.iloc[idx-1]['GPS车速']
        df['GPS车速'][idx] = np.average(df.iloc[idx-5:idx]['GPS车速'].values)
        
    return df

#### 堵车情况：连续10秒车速大于0小于10的数据，速度替换为0

In [6]:
def updateTrafficJamDate(df):
    '''
    @func: 对连续10秒车速大于0小于10的数据，速度替换为0
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''    
    # 先查出这样的数据
    num = 0
    less_10_stack = []
    for idx in range(len(df)):
        if (df.iloc[idx]['GPS车速'] < 10 and df.iloc[idx]['GPS车速'] > 0):
            less_10_stack.append(idx) # 进栈
            num += 1
        elif num < 10 and num > 0 :  
            del less_10_stack[-num:]  # 对不满足10s的已进栈数据进行退栈操作
            num = 0
        else:
            num = 0

    # 对这些数据速度全部替换为0
    for idx in tqdm(less_10_stack, desc='updateTrafficJamDate'):
        df['GPS车速'][idx] = 0   # 不知道为何，这个速度特别慢，比前面遍历一遍df还慢得多的多

    return df

#### 怠速时间超出180秒且车速为0的情况

In [7]:
def more_180_Seconds_process(df):
    '''
    @func: 选取连续超出180秒车速为0的数据，保留后180秒数据，剔除前面的，即删减冗余数据
    @params: [df] 预训练数据
    @return: [df] 处理后数据
    '''  
    def getIndex(more_180_stack,num):
        '''
        @func: 获取怠速栈中后180s的头索引
        @params: [more_180_stack] 连续怠速180以上的索引栈
        @params: [num] 连
        @return: [df] 处理后数据
        '''
        time_len = 0
        for idx,time_span in more_180_stack:
            time_len += time_span
            if time_len > num-180: return idx

    num = 0 # 记录当前栈内的时长（总的时间间隔）
    more_180_stack = []
    sum_num = 0  # 记录总的删减行数
    num_len = 0  # 记录每次删减的数据行数
    max_epoch = len(df) - 1 # 因为要删减数据，所以设置一个最大轮数
    for idx in tqdm(range(len(df)),desc='more_180_Seconds_process'):
        idx = idx - sum_num
        if idx >= (max_epoch - sum_num): break   # 数组越界处理
#         print('idx:', idx, '   ', 'num:',num,'   ', 'num_len:',num_len,'   ', 'sum_num:',sum_num)
        if df.iloc[idx]['GPS车速'] == 0:  # 进栈
            time_span = round(df.iloc[idx]['时间间隔'],0)
            if time_span > 60: # 如果怠速栈遇到时间间隔>60的，则清空栈，重新寻找（因为最终要生成的是运动学片段）
                more_180_stack = []
                num = 0
                continue
            more_180_stack.append((idx, time_span))
            num += time_span
        elif num > 180: # 大于180，寻找索引，删除数据
            begin_idx = more_180_stack[0][0]
            end_idx = getIndex(more_180_stack, num)
            df = df.drop([i for i in range(begin_idx,end_idx)])
            num_len = end_idx - begin_idx
            sum_num += num_len 
            more_180_stack = []
            num = 0
        else: # 小于180，清空栈
            more_180_stack = []
            num = 0
    return df

## 特征建立

特征参数体系构建
    1. 怠速时间   
    2. 加速时间
    3. 匀速时间
    4. 减速时间
    5. 运行时间   # 一个数据相当于一个时间点数，时间是记录整个运动片段的点数即可
    6. 运行距离
    7. 最大速度
    
    8. 匀速时间比例
    9. 怠速时间比例
    10. 减速时间比例
    11. 加速时间比例
    
    12. 速度在0-10的比例
    13. 速度在10-20的比例
    14. 速度在20-30的比例
    15. 速度在30-40的比例
    16. 速度在40-50的比例
    17. 速度在50-60的比例
    18. 速度在60-70的比例
    19. 速度在大于70的比例
    
    20. 平均速度
    21. 平均加速度
    22. 平均减速度
    23. 速度标准差
    24. 加速度标准差

> 1~7和12~19需要在实时建立，8~11可以后添

In [8]:
def creatFeatureDateframe(df, begin_index, end_index, flag):
    '''
    @func: 根据所获得的运动学片段索引，生成一条运动学记录
    @params: [df] 预训练数据
    @params: [begin_index] 运动学片段的头指针
    @params: [end_index] 运动学片段的尾指针
    @params: [flag] 标志字符，标识是创建表头还是生成数据
    @return: [new_df] 处理后数据
    '''
    new_df = pd.DataFrame(columns=['怠速时间', '加速时间', '匀速时间', '减速时间','运行时间','运行距离',
                                   '最大速度','平均速度','平均加速度','平均减速度','速度标准差','加速度标准差',
                                  '怠速时间比例', '加速时间比例', '匀速时间比例', '减速时间比例',
                                  'v_0_10_ratio','v_10_20_ratio','v_20_30_ratio','v_30_40_ratio',
                                  'v_40_50_ratio','v_50_60_ratio','v_60_70_ratio','more_70_v_ratio'])
    
    if flag: return new_df  # 创建表头数据
    
    time_run = end_index - begin_index + 1  # 总的运行时间
    num_time_idling = 0  # 怠速点数记录
    num_time_acc = 0     # 加速点数记录
    num_time_ave = 0     # 匀速点数记录
    num_time_dec = 0     # 减速点数记录
    max_v = 0   # 最大速度
    sum_v = 0   # 片段中所有速度之和
    sum_a_v = 0  # 所有加速阶段的加速度之和
    sum_d_v = 0  # 所有减速阶段的减速度之和
    sum_std_v = 0 # 速度与均速的之差的平方和
    sum_a_2_v = 0 # 加速度平方和
    v_0_10_num = 0  # 速度数值记录
    v_10_20_num = 0
    v_20_30_num = 0
    v_30_40_num = 0
    v_40_50_num = 0
    v_50_60_num = 0
    v_60_70_num = 0
    more_70_v_num = 0 
    
    for idx in range(begin_index, end_index+1):
        
        if df.iloc[idx]['加速度'] > 0.1:
            num_time_acc += 1
            sum_a_v += df.iloc[idx]['加速度']
            sum_a_2_v += df.iloc[idx]['加速度'] **2
        elif df.iloc[idx]['加速度'] < -0.1:
            num_time_dec += 1
            sum_d_v += df.iloc[idx]['加速度']
        elif df.iloc[idx]['加速度'] == 0:
            num_time_idling += 1
        else:
            num_time_ave += 1
            
        if max_v < df.iloc[idx]['GPS车速']: max_v = df.iloc[idx]['GPS车速']
        sum_v += df.iloc[idx]['GPS车速']
        
        if (df.iloc[idx]['GPS车速'] < 10 and df.iloc[idx]['GPS车速'] > 0):
            v_0_10_num += 1
        elif (df.iloc[idx]['GPS车速'] < 20 and df.iloc[idx]['GPS车速'] > 10):
            v_10_20_num += 1
        elif (df.iloc[idx]['GPS车速'] < 30 and df.iloc[idx]['GPS车速'] > 20):
            v_20_30_num += 1
        elif (df.iloc[idx]['GPS车速'] < 40 and df.iloc[idx]['GPS车速'] > 30):
            v_30_40_num += 1
        elif (df.iloc[idx]['GPS车速'] < 50 and df.iloc[idx]['GPS车速'] > 40):
            v_40_50_num += 1
        elif (df.iloc[idx]['GPS车速'] < 60 and df.iloc[idx]['GPS车速'] > 50):
            v_50_60_num += 1
        elif (df.iloc[idx]['GPS车速'] < 70 and df.iloc[idx]['GPS车速'] > 60):
            v_60_70_num += 1
        else:
            more_70_v_num += 1
            
    ave_v = sum_v / (num_time_acc + num_time_dec + num_time_ave)
    for idx in range(begin_index, end_index+1):
        sum_std_v += (df.iloc[idx]['GPS车速']-ave_v) **2
    
    # 添加数值梳理
    time_idling = num_time_idling
    time_acc = num_time_acc
    time_ave = num_time_ave
    time_dec = num_time_dec
    time_run = time_run
    len_run = sum_v / 3.6
    max_v = max_v
    ave_v = sum_v / (num_time_acc + num_time_dec + num_time_ave)
    if num_time_acc == 0: 
        ave_a_v = sum_a_2_v / (num_time_acc + 1)
    else:
        ave_a_v = sum_a_2_v / num_time_acc
    if num_time_dec == 0: 
        ave_d_v = sum_d_v / (num_time_dec + 1)
    else:
        ave_d_v = sum_d_v / num_time_dec
    std_v = (sum_std_v / (time_run - 1)) **0.5
    std_a_v = (sum_a_2_v / (num_time_acc - 1)) **0.5
    time_idling_rate = time_idling / time_run
    time_acc_rate = time_acc / time_run
    time_ave_rate = time_ave / time_run
    time_dec_rate = time_dec / time_run
    v_0_10_ratio = v_0_10_num / time_run
    v_10_20_ratio = v_10_20_num / time_run
    v_20_30_ratio = v_20_30_num / time_run
    v_30_40_ratio = v_30_40_num / time_run
    v_40_50_ratio = v_40_50_num / time_run
    v_50_60_ratio = v_50_60_num / time_run
    v_60_70_ratio = v_60_70_num / time_run
    more_70_v_ratio = more_70_v_num / time_run
    
    
    # 添加运动学片段数据
    new_df = new_df.append({'怠速时间':time_idling, 
                            '加速时间':time_acc, 
                            '匀速时间':time_ave, 
                            '减速时间':time_dec,
                            '运行时间':time_run,
                            '运行距离':len_run,
                            '最大速度':max_v,
                            '平均速度':ave_v,
                            '平均加速度':ave_a_v,
                            '平均减速度':ave_d_v,
                            '速度标准差':std_v,
                            '加速度标准差':std_a_v,
                            '怠速时间比例':time_idling_rate,
                            '加速时间比例':time_acc_rate, 
                            '匀速时间比例':time_ave_rate,
                            '减速时间比例':time_dec_rate,
                            'v_0_10_ratio':v_0_10_ratio,
                            'v_10_20_ratio':v_10_20_ratio,
                            'v_20_30_ratio':v_20_30_ratio,
                            'v_30_40_ratio':v_30_40_ratio,
                            'v_40_50_ratio':v_40_50_ratio,
                            'v_50_60_ratio':v_50_60_ratio,
                            'v_60_70_ratio':v_60_70_ratio,
                            'more_70_v_ratio':more_70_v_ratio},ignore_index=True)
    return new_df
    
    

def searctKinematicSegment(df):
    '''
    @func: 在训练数据中搜索合适的运动学片段，让后创建新的数据表
    @params: [df] 预训练数据
    @return: [feature_df] 生成的运动学片段新数据表
    '''
    begin_index, end_index = 0, 0
    pre_v = 0  # 上一个运动点的速度
    tag = 0 # 0表示新的运动学片段还没开始，遇0则开始， 1表示运动已开始，遇1也是继续循环
    feature_df = creatFeatureDateframe(df, 0, 0, 1)  # 创建表头
    feature_idx = 0
    for idx in tqdm(range(len(df)),desc='searctKinematicSegment'):
        # 如果时间间隔 > 60秒，则放弃正在进行的运动学片段，开始重新寻找新的运动学片段
        if idx == 0:
            interval = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S') \
                            - datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        else:
            interval = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S') \
                            - datetime.datetime.strptime(df.iloc[idx-1]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
        if(int(''.join(re.findall(r"\d+\.?\d*",str(interval)))) > 60):
            pre_v = df.iloc[idx]['GPS车速']
            tag = 0
        
        # 运动学片段的判定部分
        if(df.iloc[idx]['GPS车速'] == 0 and tag == 1):  
            if pre_v!=0:  # 则构成了一个运动学片段
                end_index = idx
                now_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
                time_span = now_time - last_time
                time_span = int(''.join(re.findall(r"\d+\.?\d*",str(time_span))))
                if time_span > 20:  # 对运动时间在20秒以上的，创建运动学片段数据，并添加
                    new_df = creatFeatureDateframe(df, begin_index, end_index, 0)
                    feature_df = feature_df.append(new_df, ignore_index = True)
                    feature_idx += 1
                else:
                    begin_index = idx
                    last_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
                tag = 0  
                pre_v = df.iloc[idx]['GPS车速']
        elif (df.iloc[idx]['GPS车速'] == 0 and tag == 0):  # 新的运动学片段开始
            begin_index = idx
            last_time = datetime.datetime.strptime(df.iloc[idx]['时间'][:-5],'%Y/%m/%d %H:%M:%S')
            pre_v = df.iloc[idx]['GPS车速']
            tag = 1
        elif (df.iloc[idx]['GPS车速'] != 0 and tag == 0):  # 特殊情况，继续循环
            pre_v = df.iloc[idx]['GPS车速']
        else:
            pre_v = df.iloc[idx]['GPS车速']
            
#         print('idx:', idx, '  ', 'tag:', tag, '  ', '车速:', df.iloc[idx]['GPS车速'],'  ', 'pre_v:', pre_v,
#               '  ', 'feature_idx:', feature_idx,'  ', 'begin:', begin_index, '  ', 'end:', end_index,
#               '  ', '时间间隔:', df.iloc[idx]['时间间隔'])

            
    return feature_df

## 主函数

In [9]:
def main(df,flag):
    '''
    @func: 主函数，调用以上函数，并返回运动学片段数据
    @params: [df] 预训练数据
    @params: [flag] 所处理文件的标志
    @return: [feature_df] 运动学片段数据
    ''' 
    df = fillData(df)
    df = moreAcce_and_lessDec(df)
    df = updateTrafficJamDate(df)
    df = more_180_Seconds_process(df)
    df.to_excel('./preprocess_' + str(flag) + '.xlsx')
    feature_df = searctKinematicSegment(df)
    feature_df = feature_df[feature_df['怠速时间']<180]
    return feature_df

In [10]:
raw_data_3_df.rename({'GPS车速（km/h）':'GPS车速'},inplace=True,axis=1)

feature_1 = main(raw_data_1_df, 1)
feature_2 = main(raw_data_2_df, 2)
feature_3 = main(raw_data_3_df, 3)

feature_1.to_excel('./feature_1.xlsx')
feature_2.to_excel('./feature_2.xlsx')
feature_3.to_excel('./feature_3.xlsx')

searctKinematicSegment:   4%|▍         | 8466/189234 [00:18<04:43, 637.20it/s]


searctKinematicSegment:  10%|▉         | 18879/189234 [00:38<02:40, 1060.51it/s]


searctKinematicSegment:  15%|█▌        | 29201/189234 [00:57<05:37, 473.77it/s]


searctKinematicSegment:  21%|██        | 38804/189234 [01:17<02:55, 858.36it/s]


searctKinematicSegment:  26%|██▌       | 49405/189234 [01:32<01:57, 1187.89it/s]


...

searctKinematicSegment:  73%|███████▎  | 125290/171142 [03:47<01:56, 394.89it/s]


searctKinematicSegment:  73%|███████▎  | 125369/171142 [03:47<01:58, 386.31it/s]


searctKinematicSegment:  73%|███████▎  | 125508/171142 [03:47<01:32, 492.81it/s]


searctKinematicSegment:  73%|███████▎  | 125596/171142 [03:47<01:58, 383.29it/s]


searctKinematicSegment:  73%|███████▎  | 125720/171142 [03:47<01:33, 483.29it/s]


searctKinematicSegment:  74%|███████▎  | 125829/171142 [03:48<02:17, 328.68it/s]


searctKinematicSegment:  74%|███████▎  | 125977/171142 [03:48<02:07, 353.71it/s]


searctKinematicSegment:  74%|███████▎  | 126102/171142 [03:48<01:39, 450.46it/s]


searctKinematicSegment:  74%|███████▍  | 126226/171142 [03:49<01:20, 556.37it/s]


searctKinematicSegment:  74%|███████▍  | 126351/171142 [03:49<01:07, 666.73it/s]


searctKinematicSegment:  74%|███████▍  | 126473/171142 [03:49<00:57, 770.19it/s]


searctKinematicSegment:  74%|███████▍  | 126581/171142 [03:49<01:20, 553.73it/s]


sear

searctKinematicSegment:  79%|███████▉  | 135550/171142 [04:05<01:23, 425.01it/s]


searctKinematicSegment:  79%|███████▉  | 135721/171142 [04:05<01:04, 548.55it/s]


searctKinematicSegment:  79%|███████▉  | 135845/171142 [04:05<00:53, 658.31it/s]


searctKinematicSegment:  79%|███████▉  | 135969/171142 [04:05<00:45, 765.95it/s]


searctKinematicSegment:  80%|███████▉  | 136094/171142 [04:05<00:40, 865.42it/s]


searctKinematicSegment:  80%|███████▉  | 136205/171142 [04:05<00:39, 886.67it/s]


searctKinematicSegment:  80%|███████▉  | 136311/171142 [04:06<00:58, 598.51it/s]


searctKinematicSegment:  80%|███████▉  | 136432/171142 [04:06<00:49, 704.69it/s]


searctKinematicSegment:  80%|███████▉  | 136556/171142 [04:06<00:42, 809.49it/s]


searctKinematicSegment:  80%|███████▉  | 136660/171142 [04:06<01:16, 452.17it/s]


searctKinematicSegment:  80%|███████▉  | 136740/171142 [04:07<01:23, 411.52it/s]


searctKinematicSegment:  80%|███████▉  | 136806/171142 [04:07<01:19, 432.44it/s]


sear

searctKinematicSegment:  85%|████████▌ | 145891/171142 [04:24<01:11, 354.65it/s]


searctKinematicSegment:  85%|████████▌ | 145972/171142 [04:25<01:10, 358.81it/s]


searctKinematicSegment:  85%|████████▌ | 146101/171142 [04:25<00:54, 457.73it/s]


searctKinematicSegment:  85%|████████▌ | 146172/171142 [04:25<01:08, 363.89it/s]


searctKinematicSegment:  85%|████████▌ | 146287/171142 [04:25<01:09, 359.08it/s]


searctKinematicSegment:  86%|████████▌ | 146393/171142 [04:26<01:07, 369.24it/s]


searctKinematicSegment:  86%|████████▌ | 146514/171142 [04:26<01:06, 367.76it/s]


searctKinematicSegment:  86%|████████▌ | 146699/171142 [04:26<00:50, 483.86it/s]


searctKinematicSegment:  86%|████████▌ | 146788/171142 [04:27<01:22, 295.12it/s]


searctKinematicSegment:  86%|████████▌ | 146929/171142 [04:27<01:02, 386.73it/s]


searctKinematicSegment:  86%|████████▌ | 147052/171142 [04:27<00:49, 486.66it/s]


searctKinematicSegment:  86%|████████▌ | 147176/171142 [04:27<00:40, 594.84it/s]


sear

searctKinematicSegment:  91%|█████████ | 156096/171142 [04:44<00:40, 368.59it/s]


searctKinematicSegment:  91%|█████████▏| 156238/171142 [04:44<00:38, 382.52it/s]


searctKinematicSegment:  91%|█████████▏| 156290/171142 [04:44<00:36, 410.01it/s]


searctKinematicSegment:  91%|█████████▏| 156438/171142 [04:44<00:28, 523.21it/s]


searctKinematicSegment:  91%|█████████▏| 156518/171142 [04:45<00:40, 357.37it/s]


searctKinematicSegment:  92%|█████████▏| 156613/171142 [04:45<00:42, 344.35it/s]


searctKinematicSegment:  92%|█████████▏| 156805/171142 [04:45<00:31, 456.60it/s]


searctKinematicSegment:  92%|█████████▏| 156900/171142 [04:46<00:41, 339.20it/s]


searctKinematicSegment:  92%|█████████▏| 156973/171142 [04:46<00:50, 282.77it/s]


searctKinematicSegment:  92%|█████████▏| 157039/171142 [04:46<00:41, 340.87it/s]


searctKinematicSegment:  92%|█████████▏| 157099/171142 [04:46<00:39, 359.24it/s]


searctKinematicSegment:  92%|█████████▏| 157196/171142 [04:46<00:31, 442.50it/s]


sear

searctKinematicSegment:  97%|█████████▋| 165277/171142 [05:05<00:13, 439.74it/s]


searctKinematicSegment:  97%|█████████▋| 165373/171142 [05:06<00:14, 411.37it/s]


searctKinematicSegment:  97%|█████████▋| 165492/171142 [05:06<00:15, 354.19it/s]


searctKinematicSegment:  97%|█████████▋| 165567/171142 [05:06<00:14, 373.91it/s]


searctKinematicSegment:  97%|█████████▋| 165690/171142 [05:06<00:11, 472.48it/s]


searctKinematicSegment:  97%|█████████▋| 165798/171142 [05:07<00:13, 393.28it/s]


searctKinematicSegment:  97%|█████████▋| 165955/171142 [05:07<00:10, 507.14it/s]


searctKinematicSegment:  97%|█████████▋| 166046/171142 [05:07<00:15, 333.55it/s]


searctKinematicSegment:  97%|█████████▋| 166115/171142 [05:07<00:13, 382.51it/s]


searctKinematicSegment:  97%|█████████▋| 166284/171142 [05:08<00:09, 497.83it/s]


searctKinematicSegment:  97%|█████████▋| 166381/171142 [05:08<00:14, 319.75it/s]


searctKinematicSegment:  97%|█████████▋| 166511/171142 [05:08<00:11, 413.04it/s]


sear